# Demo for Overlandflow Simulation

## Simulation Parameters

In [ ]:
# terrain info
terrain = {
    "grid_file": "spruce_watershed.txt",  # DEM file name (ASCII format)
    "nodata_value": -9999,  # nodata value in the DEM file
    "outlet_id": -1  # set as -1 if outlet id number is unknown
}

In [ ]:
# output setting
output = {
    "output_folder" : 'output_spruce',  # output folder name
    "plot_olf": True,  # plot overland flow results
    "plot_inf": True   # plot infiltration results
}

In [ ]:
# model run setting
model_run = {
    "model_run_time": 200,  # min
    "storm_duration": 10,  # min
    "time_step": 2,  # min  set the time step to create the results
    "activate_inf":  True ,#  set as true to enable infiltration process
}

In [ ]:
# infiltration setting
infil_info = {
    "conductivity_file": '',  # set as '' if no file exist m/s
    "hydraulic_conductivity": 1.0e-7,  # m/s
    "soil_water_file": '',  # set as '' if no file exist m
    "soil_water_infiltration_depth": 1e-5,  # m
    "soil_bulk_density": 1590.0,  # kg/m3
    "rock_density": 2650.0,  # kg/m3
    "initial_soil_moisture_content": 0.15, # [0-1]
    "soil_type": "sandy loam",
    "volume_fraction_coarse_fragments": 0.2,  # [0-1]
    "coarse_sed_flag": False,
    "surface_water_minimum_depth": 1.0e-8,   # m
    "soil_pore_size_distribution_index": False,
    "soil_bubbling_pressure": False,  # m
    "wetting_front_capillary_pressure_head": False,  # m
}

# supported soil type:
# 'sand', loamy sand', 'sandy loam', 'loam', 'silt loam', 'sandy clay loam',
# 'clay loam', 'silty clay loam', 'sandy clay', 'silty clay', 'clay'

In [ ]:
# overland flow setting
olf_info = {
    "rain_file": '',  # set as '' if no file exist, mm/hr
    "rain_intensity": 59.2,  # mm/hr
    "surface_water_file": '',  # set as '' if no file exist, m
    "surface_water_depth": 1e-12,  # m
    "steep_slopes": True,
    "alpha": 0.7,  # time step coefficient
    "mannings_n": 0.03,  # manning's roughness coefficient
    "g": 9.80665,  # acceleration due to gravity m/s^2
    "theta": 0.8   # wighting factor
}

# TODO: assign different rain intensity scenario for real case (2013, 2021) using observation data and rain intensity map

## Input DEM 

In [ ]:
import matplotlib.pyplot as plt
import rasterio

In [ ]:
# Load DEM file 
with rasterio.open(terrain["grid_file"]) as dataset:
    # Read the first band
    raster_data = dataset.read(1)

    # Get metadata (optional)
    extent = [dataset.bounds.left, dataset.bounds.right, dataset.bounds.bottom, dataset.bounds.top]

# Plot the raster
plt.figure(figsize=(9, 4))
plt.imshow(raster_data, cmap="viridis", extent=extent, vmin=0)
plt.colorbar(label="elevation (m)")
plt.title("Watershed DEM of the Study Area")
plt.xlabel("X")
plt.ylabel("Y")

In [ ]:
# hydraulic conductivity
with rasterio.open("hydr_cond_wildfire.tif") as dataset:
    # Read the first band
    raster_data = dataset.read(1)

    # Get metadata (optional)
    extent = [dataset.bounds.left, dataset.bounds.right, dataset.bounds.bottom, dataset.bounds.top]

# Plot the raster
plt.figure(figsize=(8, 6))
plt.imshow(raster_data, cmap="viridis", extent=extent)
plt.colorbar(label="Pixel Values")
plt.title("Hydraulic conductivity")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

# TODO: This will be modified to use the firemap or the soilgrids data type to define the hydraulic conductivity

## Run Model

In [ ]:
from flood_simulator import FloodSimulator

In [ ]:
fs = FloodSimulator(terrain, output, model_run, infil_info, olf_info)

In [ ]:
fs.run()

## Show Results

In [ ]:
# # Make a short video
import imageio.v2 as imageio
from IPython.display import Video
import os

results_dir = os.path.join(os.getcwd(), output["output_folder"])
for name in ['flow','infil']:
    img_files = [
        os.path.join(results_dir, file)
        for file in os.listdir(results_dir)
        if name in file and ".png" in file
    ]
    img_files.sort(key=lambda x: os.path.getmtime(x))
    
    with imageio.get_writer(
        os.path.join(results_dir, f"{name}.mp4"), mode="I", fps=1
    ) as writer:
        for f in img_files:
            im = imageio.imread(os.path.join(results_dir, f))
            writer.append_data(im)
    
    writer.close()

In [ ]:
# Display the overland flow results
Video(os.path.join(output["output_folder"], "flow.mp4"), embed=True, width=900, height=700)

In [ ]:
# Display the infiltration results
Video(os.path.join(output["output_folder"], "infil.mp4"), embed=True, width=900, height=700)